In [1]:
from numpy import array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import *
from tensorflow.keras.layers import Activation, Dropout, Dense,Conv2D,MaxPooling2D
from tensorflow.keras.layers import Flatten, LSTM, Concatenate
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [3]:
#For error = TensorFlow: Blas GEMM launch failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print('physical_devices : ', physical_devices)

physical_devices :  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Many-to-Many Sequence Problems
Encoder-Decoder Model

In [3]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

In [4]:
DROPOUT = 0.3
INPUT_IMAGE_SIZE=512

In [37]:
def get_model(image_size):
    inputs = Input((image_size,image_size,1))
    #512
    convL=Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    convL=Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    convL = MaxPooling2D(pool_size=(2, 2))(convL)
    #256
    convL=Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    convL=Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    convL = MaxPooling2D(pool_size=(2, 2))(convL)
    #128
    convL=Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    convL=Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    convL = MaxPooling2D(pool_size=(2, 2))(convL)
    #64
    #convL=Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL = MaxPooling2D(pool_size=(2, 2))(convL)
    #32
    #convL=Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL=Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    #convL = MaxPooling2D(pool_size=(2, 2))(convL)
    #16
    
    convL = Conv2D(1, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(convL)
    denL = Flatten()(convL)
    denL = Dense(100, activation="relu")(denL)
    lstmL = LSTM(100, activation='relu')(denL)
    
    revL = RepeatVector(3)(lstmL)
    
    outl = LSTM(100, activation='relu', return_sequences=False)(revL)
    
    outL = TimeDistributed(Dense(133))(outL)

    model = Model(inputs=inputs, outputs=ul_conv_0)
    
    model.compile(optimizer = Adam(lr = 1e-6, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-8), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    return model
    
    
    
    

In [38]:
m = get_model(512)

ValueError: Input 0 of layer lstm_2 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 100]

In [5]:
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed

model = Sequential()

# encoder layer
model.add(LSTM(100, activation='relu', input_shape=(3, 1)))

# repeat vector
model.add(RepeatVector(3))

# decoder layer
model.add(LSTM(100, activation='relu', return_sequences=True))

model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 3, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 100)            80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 1)              101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000
6/6 [==============================] - 0s 64ms/step - loss: 20144.2070 - val_loss: 54102.7227
Epoch 2/1000
6/6 [==============================] - 0s 14ms/step - loss: 13520.2852 - val_loss: 30917.6211
Epoch 3/1000
6/6 [==============================] - 0s 13ms/step - loss: 7062.9893 - val_loss: 18611.1133
Epoch 4/1000
6/6 [==============================] - 0s 14ms/step - loss: 5443.1338 - val_loss: 17794.2969
Epoch 5/1000
6/6 [==============================] - 0s 14ms/step - loss: 4465.5840 - val_loss: 13957.1055
Epoch 6/1000
6/6 [==============================] - 0s 13ms/step - loss: 3826.3037 - val_loss: 11881.8965
Epoch 7/1000
6/6 [==============================] - 0s 14ms/step - loss: 2950.1013 - val_loss: 8367.1650
Epoch 8/1000
6/6 [==============================] - 0s 14ms/step - loss: 2163.9731 - val_loss: 5820.0254
Epoch 9/1000
6/6 [==============================] - 0s 14ms/step - loss: 1232.0833 - val_loss: 2663.0906
Epoch 10/1000
6/6 [============================

Epoch 82/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.0412 - val_loss: 12.3752
Epoch 83/1000
6/6 [==============================] - 0s 15ms/step - loss: 1.0708 - val_loss: 7.3906
Epoch 84/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.7792 - val_loss: 1.3213
Epoch 85/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.9900 - val_loss: 1.9012
Epoch 86/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.6479 - val_loss: 15.3336
Epoch 87/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.7835 - val_loss: 3.8296
Epoch 88/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.6588 - val_loss: 1.6724
Epoch 89/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.4816 - val_loss: 5.0156
Epoch 90/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.2296 - val_loss: 2.9879
Epoch 91/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.6194 - val_loss: 1.386

6/6 [==============================] - 0s 16ms/step - loss: 11.0643 - val_loss: 87.4735
Epoch 244/1000
6/6 [==============================] - 0s 16ms/step - loss: 19.0313 - val_loss: 2.5259
Epoch 245/1000
6/6 [==============================] - 0s 13ms/step - loss: 8.9791 - val_loss: 78.9707
Epoch 246/1000
6/6 [==============================] - 0s 13ms/step - loss: 14.5080 - val_loss: 14.2951
Epoch 247/1000
6/6 [==============================] - 0s 14ms/step - loss: 18.5555 - val_loss: 19.5191
Epoch 248/1000
6/6 [==============================] - 0s 14ms/step - loss: 11.0128 - val_loss: 16.1336
Epoch 249/1000
6/6 [==============================] - 0s 13ms/step - loss: 2.5338 - val_loss: 64.6464
Epoch 250/1000
6/6 [==============================] - 0s 14ms/step - loss: 4.9310 - val_loss: 3.0484
Epoch 251/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.8932 - val_loss: 1.5739
Epoch 252/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.4762 - val_loss: 1

6/6 [==============================] - ETA: 0s - loss: 0.068 - 0s 14ms/step - loss: 0.0687 - val_loss: 4.6807
Epoch 406/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.5321 - val_loss: 8.8430
Epoch 407/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.9629 - val_loss: 12.4115
Epoch 408/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.6971 - val_loss: 7.7140
Epoch 409/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.0146 - val_loss: 17.4871
Epoch 410/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.5592 - val_loss: 25.8695
Epoch 411/1000
6/6 [==============================] - 0s 15ms/step - loss: 3.5784 - val_loss: 66.6750
Epoch 412/1000
6/6 [==============================] - 0s 14ms/step - loss: 9.0853 - val_loss: 52.0050
Epoch 413/1000
6/6 [==============================] - 0s 14ms/step - loss: 6.7282 - val_loss: 31.9082
Epoch 414/1000
6/6 [==============================] - 0s 14ms/step - loss: 3

6/6 [==============================] - 0s 14ms/step - loss: 3.7828 - val_loss: 17.7492
Epoch 567/1000
6/6 [==============================] - 0s 14ms/step - loss: 3.6053 - val_loss: 10.2961
Epoch 568/1000
6/6 [==============================] - 0s 13ms/step - loss: 9.8143 - val_loss: 11.8965
Epoch 569/1000
6/6 [==============================] - 0s 15ms/step - loss: 14.5906 - val_loss: 35.3653
Epoch 570/1000
6/6 [==============================] - 0s 15ms/step - loss: 6.7441 - val_loss: 0.5677
Epoch 571/1000
6/6 [==============================] - 0s 14ms/step - loss: 5.5045 - val_loss: 54.8503
Epoch 572/1000
6/6 [==============================] - 0s 15ms/step - loss: 35.1681 - val_loss: 102.6470
Epoch 573/1000
6/6 [==============================] - 0s 14ms/step - loss: 40.8709 - val_loss: 502.4495
Epoch 574/1000
6/6 [==============================] - 0s 15ms/step - loss: 69.5518 - val_loss: 342.8912
Epoch 575/1000
6/6 [==============================] - 0s 14ms/step - loss: 50.7578 - val_lo

Epoch 647/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0228 - val_loss: 0.5069
Epoch 648/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0169 - val_loss: 0.8748
Epoch 649/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0132 - val_loss: 0.3555
Epoch 650/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0118 - val_loss: 0.5378
Epoch 651/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0100 - val_loss: 0.4306
Epoch 652/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0210 - val_loss: 0.3594
Epoch 653/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0201 - val_loss: 0.0868
Epoch 654/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.1378 - val_loss: 0.6186
Epoch 655/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0817 - val_loss: 1.5598
Epoch 656/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0494 - val_los

6/6 [==============================] - 0s 14ms/step - loss: 2.6982 - val_loss: 3.4603
Epoch 729/1000
6/6 [==============================] - 0s 13ms/step - loss: 3.5388 - val_loss: 5.4974
Epoch 730/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.8226 - val_loss: 38.1004
Epoch 731/1000
6/6 [==============================] - 0s 13ms/step - loss: 1.7802 - val_loss: 14.4061
Epoch 732/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.5517 - val_loss: 22.8893
Epoch 733/1000
6/6 [==============================] - 0s 14ms/step - loss: 3.6735 - val_loss: 10.9860
Epoch 734/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.4171 - val_loss: 9.6598
Epoch 735/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.5683 - val_loss: 9.7087
Epoch 736/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.3979 - val_loss: 5.9721
Epoch 737/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.1940 - val_loss: 12.2639


Epoch 809/1000
6/6 [==============================] - 0s 14ms/step - loss: 4.5334 - val_loss: 2.6840
Epoch 810/1000
6/6 [==============================] - 0s 13ms/step - loss: 1.8052 - val_loss: 0.4886
Epoch 811/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.7320 - val_loss: 4.9108
Epoch 812/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.2526 - val_loss: 5.6128
Epoch 813/1000
6/6 [==============================] - 0s 14ms/step - loss: 2.6198 - val_loss: 3.8333
Epoch 814/1000
6/6 [==============================] - 0s 13ms/step - loss: 5.4463 - val_loss: 15.5339
Epoch 815/1000
6/6 [==============================] - 0s 14ms/step - loss: 14.4547 - val_loss: 44.4168
Epoch 816/1000
6/6 [==============================] - 0s 14ms/step - loss: 31.8247 - val_loss: 1.8500
Epoch 817/1000
6/6 [==============================] - 0s 14ms/step - loss: 12.8019 - val_loss: 8.3515
Epoch 818/1000
6/6 [==============================] - 0s 14ms/step - loss: 6.5147 - va

6/6 [==============================] - 0s 14ms/step - loss: 10.0121 - val_loss: 0.2686
Epoch 971/1000
6/6 [==============================] - 0s 14ms/step - loss: 13.2427 - val_loss: 76.8403
Epoch 972/1000
6/6 [==============================] - 0s 13ms/step - loss: 41.7494 - val_loss: 587.8608
Epoch 973/1000
6/6 [==============================] - 0s 14ms/step - loss: 172.6696 - val_loss: 47.0765
Epoch 974/1000
6/6 [==============================] - 0s 14ms/step - loss: 75.6409 - val_loss: 427.6559
Epoch 975/1000
6/6 [==============================] - 0s 14ms/step - loss: 47.0557 - val_loss: 688.0444
Epoch 976/1000
6/6 [==============================] - 0s 14ms/step - loss: 68.3552 - val_loss: 3.9594
Epoch 977/1000
6/6 [==============================] - 0s 13ms/step - loss: 62.2807 - val_loss: 109.4679
Epoch 978/1000
6/6 [==============================] - 0s 14ms/step - loss: 759.3130 - val_loss: 1340.3981
Epoch 979/1000
6/6 [==============================] - 0s 15ms/step - loss: 300.385

In [7]:
test_input = array([300.0, 305.0, 310.0])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[[315.1471 ]
  [321.728  ]
  [328.90057]]]


In [8]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu', input_shape=(3, 1))))
model.add(RepeatVector(3))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000


TypeError: in user code:

    c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\program files\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\program files\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\program files\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:927 __call__
        outputs = call_fn(c